In [ ]:
import nltk
import numpy as np
import random
import string # to process standard python strings
import os, json
import pandas as pd

In [ ]:
data = []
with open('data/climatology-1.json') as json_file:
    data = json.load(json_file)
count = 1

def traverseAnswerObjects(answersObjects):
    print("answer: " + answersObjects["text"])
    print("\n")
    if(len(answersObjects["answers"])):
        for answerObject in answersObjects["answers"]:
            traverseAnswerObjects(answerObject)
    return

for questionObject in data:
    traverseAnswerObjects(questionObject)

    count += 1
    if(count == 2):
        break